In [1]:
!python --version

Python 3.5.6 :: Anaconda, Inc.


In [2]:
import pandas as pd
import numpy as np
import matplotlib
import ast

In [3]:
tweets = pd.read_csv('../data/external/csv_datas_full.csv', sep = '\t', low_memory=False)

In [4]:
tweets[:5].head()

,tweet_id,tweet_creation_dt,tweet_source,tweet_language,tweet_text,tweet_location_coordinates,tweet_quote_count,tweet_reply_count,tweet_retweet_count,tweet_favorite_count,...,quoted_tweet_publisher_nickname,quoted_tweet_publisher_location,quoted_tweet_publisher_description,quoted_tweet_publisher_declared_language,quoted_tweet_publisher_followers_count,quoted_tweet_publisher_friends_count,quoted_tweet_publisher_listed_count,quoted_tweet_publisher_favourites_count,quoted_tweet_publisher_tweet_count,quoted_tweet_publisher_account_creation_dt
0,580141359983501312,23/03/2015 22:57:33,dlvr.it,fr,JO 2024 : nouvelle étape vers la candidature d...,NaN,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,580140029084729344,23/03/2015 22:52:16,Google,fr,http://t.co/opWYuQtXHd Hidalgo favorable à une...,NaN,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,580136498105843712,23/03/2015 22:38:14,Twitter for Android,fr,"JO 2024 : Hidalgo veut ""engager pleinement"" Pa...",NaN,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,580133490668232704,23/03/2015 22:26:17,iOS,fr,"""Hidalgo veut ""engager pleinement"" Paris vers ...",NaN,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,580130710264004608,23/03/2015 22:15:14,WordPress.com,fr,JO 2024 : nouvelle étape vers la candidature d...,NaN,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
tweets.dtypes

tweet_id                                        int64
tweet_creation_dt                              object
tweet_source                                   object
tweet_language                                 object
tweet_text                                     object
tweet_location_coordinates                     object
tweet_quote_count                               int64
tweet_reply_count                               int64
tweet_retweet_count                             int64
tweet_favorite_count                            int64
tweet_used_hashtags_list                       object
tweet_user_mentions_list                       object
publisher_id                                    int64
publisher_name                                 object
publisher_nickname                             object
publisher_location                             object
publisher_description                          object
publisher_declared_language                    object
publisher_followers_count   

In [6]:
tweets.shape

(36219, 51)

## Première statistique rapide

### Langue

In [7]:
tweets['tweet_language'].value_counts()

fr    36219
Name: tweet_language, dtype: int64

### Tweets source

In [8]:
tweets['tweet_source'].value_counts().head(20)

Twitter Web Client                  11840
Twitter for iPhone                   9621
Twitter for Android                  6459
TweetDeck                            1692
Twitter for iPad                      745
IFTTT                                 652
Hootsuite                             631
erased13906048                        597
Facebook                              470
Buffer                                349
Twitter Lite                          341
Hobbynote Engage                      225
Linkis: turn sharing into growth      167
Instagram                             165
Twitter for Windows Phone             131
LinkedIn                              127
dlvr.it                               126
twitterfeed                           118
Media Studio                          104
iOS                                    99
Name: tweet_source, dtype: int64

In [9]:
tweets['tweet_source'].value_counts().tail(20)

Adobe® Social             1
Maurice Bot               1
erased12875113            1
Automobiles Cars          1
Medias-Info WP            1
WP 974                    1
Feedly                    1
Social Dynamite           1
Talon (Plus)              1
plurielapps               1
PlurielClemence           1
Le Figaro.fr on iOS       1
KETIL IMMOBILIER RT       1
Twidere for Android #3    1
Twidere for Android #4    1
WMyWay                    1
Qui a TT?                 1
Alouette                  1
Commun.it Intelligence    1
Twittimer                 1
Name: tweet_source, dtype: int64

### Reply

In [10]:
tweets.groupby('tweet_is_reply').size()

tweet_is_reply
0    32219
1     4000
dtype: int64

### Longueur tweet

In [11]:
tweets['tweet_text'].apply(len).describe()

count    36219.000000
mean       142.055606
std         50.112228
min         17.000000
25%        116.000000
50%        136.000000
75%        157.000000
max        811.000000
Name: tweet_text, dtype: float64

In [12]:
def firstDayOfMonth(datetime):
    return datetime.date().replace(day = 1)

tweets['tweet_creation_month'] = pd.to_datetime(tweets['tweet_creation_dt']).apply(firstDayOfMonth)

In [13]:

tweets_len = tweets.groupby('tweet_creation_month')['tweet_text'].agg({
    lambda x: np.mean(x.apply(lambda x : len(x))),
    len
})


In [14]:
tweets_len.columns = ['tweet_text_longueur', 'effectif']
tweets_len.head()

,tweet_text_longueur,effectif
tweet_creation_month,,
2015-01-01,94,122.946809
2015-02-01,218,120.155963
2015-03-01,650,113.700000
2015-04-01,802,120.932668
2015-05-01,336,125.053571


In [15]:
tweets_len.plot(y = 'effectif', use_index=True)
tweets_len.plot(y = 'tweet_text_longueur', use_index=True)

### Likes

In [16]:
tweets.groupby(tweets['tweet_favorite_count'] > 0).size()

tweet_favorite_count
False    15488
True     20731
dtype: int64

In [17]:
tweets[tweets['tweet_favorite_count'] > 0]['tweet_favorite_count'].describe()

count    20731.000000
mean        15.602576
std         87.624517
min          1.000000
25%          1.000000
50%          3.000000
75%          9.000000
max       5182.000000
Name: tweet_favorite_count, dtype: float64

## Répartition des hashtags

In [18]:
hashtags = tweets['tweet_used_hashtags_list']

In [19]:
all_hastags = list()
for el in hashtags:
    el = ast.literal_eval(el)
    if len(el) > 0:
        all_hastags.extend(el)

In [20]:
hashtags = pd.DataFrame(all_hastags, columns=['hashtag'])

In [21]:
hashtags['hashtag_lower_case'] = hashtags['hashtag'].str.lower()

In [22]:
hashtags.groupby('hashtag_lower_case')['hashtag_lower_case'].count().reset_index(name = 'count').sort_values(['count'], ascending=False)

,hashtag_lower_case,count
5041,paris2024,20695
3573,jo2024,10507
5012,paris,2334
6317,sport,1652
3561,jo,1049
4804,olympicday,680
2112,enmodejo,626
2909,grandparis,607
1277,cio,499
3616,jouerlejeu,467


## Répartion des mentions

In [23]:
mentions = tweets['tweet_user_mentions_list']

In [24]:
all_mentions = list()
for el in mentions:
    el = ast.literal_eval(el)
    if len(el) > 0:
        all_mentions.extend(el)

In [25]:
mentions = pd.DataFrame(all_mentions, columns=['mention'])

In [26]:
mentions['mention_lower_case'] = mentions['mention'].str.lower()

In [27]:
mentions.groupby('mention_lower_case')['mention_lower_case'].count().reset_index(name = 'count').sort_values(['count'], ascending=False)

,mention_lower_case,count
4771,paris2024,2474
393,anne_hidalgo,1335
4765,paris,820
6073,tonyestanguet,820
1922,emmanuelmacron,378
2389,franceolympique,377
2270,flessellaura,366
6491,vpecresse,279
2917,iledefrance,243
6221,unss,191
